In [1]:
import requests 
import re
from bs4 import BeautifulSoup 
import pandas as pd
import pymongo 
from nltk.corpus import stopwords
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from config import indeed_query

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}

[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\Tim\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [3]:
stop_words = set(stopwords.words('english'))  

In [4]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn) 
db = client.indeed
collection = db.posts

base_url='https://www.indeed.com/'
qloc='Washington+DC'
query=indeed_query

indeed_query

'asp.net+-clearance'

In [6]:

for i in range(0,200,10):
    url = f"{base_url}jobs?q={query}&l={qloc}&jt=fulltime&start={i}"
    print(url)
    break;
    soup = BeautifulSoup(requests.get(url).text, "html.parser")  
    
    els = soup.select('.jobsearch-SerpJobCard') 
    if len(els) < 1:
        break
    for e in els: 
        if len(e) < 1:
            break
        dic = {}
        dic['query']=query
        dic['qloc']=qloc
        dic['title']=e.select('.title')[0].text.replace('\n','')
        dic['company']=e.select('.company')[0].text.replace('\n','')
        dic['location']=e.select('.location')[0].text.replace('\n','')
        dic['summary']=e.select('.summary')[0].text.replace('\n','')
        dic['entry']='-'.join([c for c in re.findall('\w*', f"{dic['title']}{dic['company']}") if c != '' and c not in stop_words])
        dic['date']=e.select('.date')[0].text.replace('\n','')
        if len(e.select('.salaryText')) > 0: 
            salaryText=e.select('.salaryText')[0].text.replace('\n','')
            dic['salary'] = re.findall('\d{2,3}\,\d{1,3}',salaryText) 
            if len(dic['salary']) > 1:
                dic['salarymin'] =  dic['salary'][0]
                dic['salarymax'] =  dic['salary'][1] 
        collection.delete_one(  {'entry': dic['entry']}  )
        collection.insert_one(dic) 
 


https://www.indeed.com/jobs?q=asp.net+-clearance&l=Washington+DC&jt=fulltime&start=0


In [7]:
df = pd.DataFrame(collection.find({}))
df['salarymin'] = df['salarymin'].apply(lambda x: str(x).replace(',', '')).astype('float') 
df['url'] = 'https://www.google.com/search?q=' + df['company'] + '+Careers'
df['url'] = df['url'].apply(lambda x: str(x).replace(' ', '%20'))
df['date'] = df['date'].apply(lambda x: re.sub('days ago|day ago|\+|Today|\s', '', x)).replace('','0').astype('float') 
df.sort_values(by='date', ascending=True, inplace=True) 
#df = df.loc[df.title.str.contains('Developer|Engineer|Software', regex=True)]
for i, r in df.iterrows():
    print(f"{r['title']} {r['company']} {r['url']}" )
df

UI Designer/Architectnew Distributed Solutions Inc. https://www.google.com/search?q=Distributed%20Solutions%20Inc.+Careers
.Net Developernew Bravent LLC https://www.google.com/search?q=Bravent%20LLC+Careers
Senior Solution Architectnew Distributed Solutions Inc. https://www.google.com/search?q=Distributed%20Solutions%20Inc.+Careers
Software Developer/Engineernew Distributed Solutions Inc. https://www.google.com/search?q=Distributed%20Solutions%20Inc.+Careers
Tax Transformation .Net Developer - Seniornew Deloitte https://www.google.com/search?q=Deloitte+Careers
Tax Technology Support Analystnew Deloitte https://www.google.com/search?q=Deloitte+Careers
Net Developernew SonoMax LLC https://www.google.com/search?q=SonoMax%20LLC+Careers
Junior Developernew ITCON Services https://www.google.com/search?q=ITCON%20Services+Careers
.NET Developernew NETE an NTT DATA Company https://www.google.com/search?q=NETE%20an%20NTT%20DATA%20Company+Careers
Junior Developer - Active TS/SCI with Polygraph ON

,_id,query,qloc,title,company,location,summary,entry,date,salary,salarymin,salarymax,url
183,604aa05953916a48e6ebafdb,asp.net+-clearance,Washington+DC,UI Designer/Architectnew,Distributed Solutions Inc.,"Reston, VA 20191 (Hattontown area)","Candidates must thrive in a fast-paced, agile ...",UI-Designer-ArchitectnewDistributed-Solutions-Inc,0.0,NaN,NaN,NaN,https://www.google.com/search?q=Distributed%20...
1,601f3fc9e88677d0b76e85f4,asp.net+-clearance,Washington+DC,.Net Developernew,Bravent LLC,"Washington, DC","ASP.net: 1 year (Preferred).NET Framework, ASP...",Net-DevelopernewBravent-LLC,0.0,[],NaN,NaN,https://www.google.com/search?q=Bravent%20LLC+...
210,604aa05a53916a48e6ebaff8,asp.net+-clearance,Washington+DC,Senior Solution Architectnew,Distributed Solutions Inc.,"Reston, VA 20191 (Hattontown area)",The ideal candidate will have a broad understa...,Senior-Solution-ArchitectnewDistributed-Soluti...,0.0,NaN,NaN,NaN,https://www.google.com/search?q=Distributed%20...
280,604aa06153916a48e6ebb0d0,asp.net+-clearance,Washington+DC,Software Developer/Engineernew,Distributed Solutions Inc.,"Reston, VA 20191 (Hattontown area)",Develop and maintain technical project plans t...,Software-Developer-EngineernewDistributed-Solu...,0.0,NaN,NaN,NaN,https://www.google.com/search?q=Distributed%20...
23,601f3fcae88677d0b76e861d,asp.net+-clearance,Washington+DC,Tax Transformation .Net Developer - Seniornew,Deloitte,"Suitland, MD",Net Developer you will be responsible for desi...,Tax-Transformation-Net-Developer-SeniornewDelo...,0.0,NaN,NaN,NaN,https://www.google.com/search?q=Deloitte+Careers
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,601f3fcce88677d0b76e864e,asp.net+-clearance,Washington+DC,Web Applications Developer,NEMA,"Arlington, VA 22209 (Radnor-Ft Myer Heights area)",NEMA crafts and executes advocacy campaigns an...,Web-Applications-DeveloperNEMA,30.0,NaN,NaN,NaN,https://www.google.com/search?q=NEMA+Careers
142,601f3fd3e88677d0b76e870f,asp.net+-clearance,Washington+DC,Microsoft D365 Business Central Technical Seni...,MorganFranklin Consulting,"McLean, VA",Advise fast growing companies and well-known e...,Microsoft-D365-Business-Central-Technical-Seni...,30.0,NaN,NaN,NaN,https://www.google.com/search?q=MorganFranklin...
286,604aa06153916a48e6ebb0d6,asp.net+-clearance,Washington+DC,Web Developer/Programmer (SharePoint) – Mid (T...,Eursdale Companies,"Springfield, VA",Develops technical architecture of internet we...,Web-Developer-Programmer-SharePoint-Mid-TS-SCI...,30.0,NaN,NaN,NaN,https://www.google.com/search?q=Eursdale%20Com...
198,604aa05a53916a48e6ebafec,asp.net+-clearance,Washington+DC,.NET Engineer,CapTech Consulting,"Reston, VA","Across industries and business goals, we fuse ...",NET-EngineerCapTech-Consulting,30.0,NaN,NaN,NaN,https://www.google.com/search?q=CapTech%20Cons...


In [8]:
conn = 'mongodb://localhost:27017' 
cursor = collection.find(
    { }, {'_id': False})
df = pd.DataFrame(cursor)
df

,query,qloc,title,company,location,summary,entry,date,salary,salarymin,salarymax
0,asp.net+-clearance,Washington+DC,Software Developer IVnew,"LISCR, LLC","Dulles, VA 20166",The Software Developer IV responsibilities inc...,Software-Developer-IVnewLISCR-LLC,5 days ago,NaN,NaN,NaN
1,asp.net+-clearance,Washington+DC,.Net Developernew,Bravent LLC,"Washington, DC","ASP.net: 1 year (Preferred).NET Framework, ASP...",Net-DevelopernewBravent-LLC,Today,[],NaN,NaN
2,asp.net+-clearance,Washington+DC,ASP.Net / C# Programmers,Marjen IT,"Washington, DC",NET and Visual C# in Oracle database supported...,ASP-Net-C-ProgrammersMarjen-IT,30+ days ago,NaN,NaN,NaN
3,asp.net+-clearance,Washington+DC,.Net Developer,FAAZ Consulting,"Washington, DC","Competence in writing clean, bug-free code as ...",Net-DeveloperFAAZ-Consulting,15 days ago,"[86,205, 142,944]","86,205","142,944"
4,asp.net+-clearance,Washington+DC,ASP.NET Developer,Sriram Business Solutions,"Washington, DC","Position requires a Bachelor Degree, 3-5 years...",ASP-NET-DeveloperSriram-Business-Solutions,30+ days ago,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
283,asp.net+-clearance,Washington+DC,IT Specialist (Application Software),Office of the Chief Financial Officer (OCFO),"Washington, DC 20024 (Southwest Waterfront area)",OF VACANCIES: One (1) or moreAGENCY: Office of...,IT-Specialist-Application-Software-Office-Chie...,30+ days ago,NaN,NaN,NaN
284,asp.net+-clearance,Washington+DC,Senior Database Developer (CSSA),Broadleaf-inc,"Washington, DC 20310",The Database Administrator - Senior independen...,Senior-Database-Developer-CSSA-Broadleaf-inc,30+ days ago,NaN,NaN,NaN
285,asp.net+-clearance,Washington+DC,Senior Full Stack Angular/.NET Software Engineer,REI Systems,"Sterling, VA 20166",Be an integral part of a scrum team participat...,Senior-Full-Stack-Angular-NET-Software-Enginee...,23 days ago,NaN,NaN,NaN
286,asp.net+-clearance,Washington+DC,Web Developer/Programmer (SharePoint) – Mid (T...,Eursdale Companies,"Springfield, VA",Develops technical architecture of internet we...,Web-Developer-Programmer-SharePoint-Mid-TS-SCI...,30+ days ago,NaN,NaN,NaN


In [9]:
 
collection.delete_many(
     { 'entry': {'$regex': '^.*$', '$options': 'i'}}
)
 

In [10]:
browser = Browser('chrome', **executable_path, headless=False)

base_url='https://www.indeed.com/'
qloc='Washington+DC'
query='junior+data+python'
for i in range(0,20,10): 
    browser.visit(f"{base_url}jobs?q={query}&l={qloc}&jt=fulltime&start={i}")
    soup = BeautifulSoup( browser.html , 'html.parser') 
    for ele in soup.select('.jobsearch-SerpJobCard a.jobtitle'): 
        browser.click_link_by_id(f"{ele['id']}")

        with browser.get_iframe('vjs-container-iframe') as iframe: 
            soup = BeautifulSoup( iframe.html , 'html.parser')
            print(soup.select('#jobDescriptionText') == None)  

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
